In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from textblob import TextBlob
from tweepy import Stream
from bs4 import BeautifulSoup
import requests
import pprint
import json
import pandas as pd
import pymongo
from sqlalchemy import func
from nltk.stem import WordNetLemmatizer
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
from sklearn.ensemble import RandomForestClassifier
# nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

C:\Users\sande\Anaconda3\envs\pythondata\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define the 'classDB' database in Mongo
db = client.projDB
tweet = db.char_hashtags.find()

In [3]:
df = pd.DataFrame(columns=["character_lower", "character", "id", "text", "language","created", "place", "source", "sentiment", "polarity", "subjectivity", "Preprocess"])
df1 = df

In [4]:
count = 0
for each_tweet in tweet:
#     print(each_tweet)
#     print("")
#     print (each_tweet["id"])
#     print(each_tweet["text"])
    analysis = TextBlob(each_tweet["text"])
#     print(analysis.sentiment)
    senti = ""
    polarity = 0.0
    subjectivity = 0.0
    
    if analysis.sentiment[0]>0:
#        print("pos")
       senti = "Pos"
    elif analysis.sentiment[0]<0:
#        print("neg")
       senti = "Neg"
    else:
#        print("neu")
       senti = "Neu"
    
    polarity = float(analysis.sentiment[0])
    subjectivity = float(analysis.sentiment[1])
    
#     name = each_tweet["entities"]
#     print(each_tweet["entities"])
#     print(senti)
#     print("")
#     print(each_tweet["entities"]["hashtags"][0]["text"])
    if each_tweet["entities"]["hashtags"] is None:
        char_name = ""
    else:
        if len(each_tweet["entities"]["hashtags"]) > 0:
            char_name = each_tweet["entities"]["hashtags"][0]["text"]
    
#     print(char_name)
    try: 
        df1 = pd.DataFrame({"character_lower": char_name.lower(), "character": char_name, "id": each_tweet["id"], "text": each_tweet["text"], "language": each_tweet["language"], "created": each_tweet["created"], "place": each_tweet["place"], "source": each_tweet["source"], "sentiment": [senti], "polarity": [polarity], "subjectivity": [subjectivity], "Preprocess": ""})
        df = df.append(df1, ignore_index=True)
    except:
#         print(each_tweet)
#         print(f'{char_name}, {each_tweet["id"]}, {each_tweet["text"]}, {each_tweet["language"]}, {each_tweet["created"]}, {each_tweet["place"]}, {each_tweet["source"]}, {[senti]}')
        pass
    count += 1
    if count > 100:
        break

In [5]:
df_backup = df
# df_backup.head()

In [6]:
df.head()

,character_lower,character,id,text,language,created,place,source,sentiment,polarity,subjectivity,Preprocess
0,gameofthrones,gameofthrones,1128107890034737152,RT @kbrxci: Look at their faces...They've warn...,en,Tue May 14 01:20:54 +0000 2019,None,"<a href=""http://twitter.com/download/iphone"" r...",Neu,0.0,0.0,
1,gameofthrones,GameofThrones,1128107890076790789,RT @isthatdejavuu: Sansa in Winterfell watchin...,en,Tue May 14 01:20:54 +0000 2019,None,"<a href=""http://twitter.com/download/iphone"" r...",Neg,-0.2,0.8,
2,gameofthrones,GameofThrones,1128107890479378432,RT @moashrav: Spoilers without context\n#Gameo...,en,Tue May 14 01:20:54 +0000 2019,None,"<a href=""http://twitter.com/download/iphone"" r...",Neu,0.0,0.0,
3,gameofthrones,GameofThrones,1128107890731102210,RT @CriticalBard: So... you’re telling me... s...,en,Tue May 14 01:20:55 +0000 2019,None,"<a href=""http://twitter.com/download/android"" ...",Neu,0.0,0.0,
4,madqueen,MadQueen,1128107891322380288,RT @HbWINGer27: A close up of Varys' letter ab...,en,Tue May 14 01:20:55 +0000 2019,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Neu,0.0,0.0,


In [7]:
stemmer = WordNetLemmatizer()

for index, row in df.iterrows():
    # Remove all the special characters
#     print(row["text"])
    test_preproc = re.sub(r'\W', ' ', str(row["text"]))

    # remove all single characters
    test_preproc = re.sub(r'\s+[a-zA-Z]\s+', ' ', test_preproc)

    # Remove single characters from the start
    test_preproc = re.sub(r'\^[a-zA-Z]\s+', ' ', test_preproc) 

    # Substituting multiple spaces with single space
    test_preproc = re.sub(r'\s+', ' ', test_preproc, flags=re.I)

    # Removing prefixed 'b'
    test_preproc = re.sub(r'^b\s+', '', test_preproc)

    # Converting to Lowercase
    test_preproc = test_preproc.lower()
    
    # Lemmatization
    test_preproc = test_preproc.split()

    test_preproc = [stemmer.lemmatize(word) for word in test_preproc]
#     print(test_preproc)
    comb_string = ' '.join(test_preproc)
    df.loc[index, "Preprocess"] = comb_string
#     row["Preprocess"] = str(comb_string)
#     print(comb_string)
#     print(row["Preprocess"])
#     print("")

In [8]:
# df["character_lower"].unique()
df.head()

,character_lower,character,id,text,language,created,place,source,sentiment,polarity,subjectivity,Preprocess
0,gameofthrones,gameofthrones,1128107890034737152,RT @kbrxci: Look at their faces...They've warn...,en,Tue May 14 01:20:54 +0000 2019,None,"<a href=""http://twitter.com/download/iphone"" r...",Neu,0.0,0.0,rt kbrxci look at their face they ve warned u ...
1,gameofthrones,GameofThrones,1128107890076790789,RT @isthatdejavuu: Sansa in Winterfell watchin...,en,Tue May 14 01:20:54 +0000 2019,None,"<a href=""http://twitter.com/download/iphone"" r...",Neg,-0.2,0.8,rt isthatdejavuu sansa in winterfell watching ...
2,gameofthrones,GameofThrones,1128107890479378432,RT @moashrav: Spoilers without context\n#Gameo...,en,Tue May 14 01:20:54 +0000 2019,None,"<a href=""http://twitter.com/download/iphone"" r...",Neu,0.0,0.0,rt moashrav spoiler without context gameofthro...
3,gameofthrones,GameofThrones,1128107890731102210,RT @CriticalBard: So... you’re telling me... s...,en,Tue May 14 01:20:55 +0000 2019,None,"<a href=""http://twitter.com/download/android"" ...",Neu,0.0,0.0,rt criticalbard so you re telling me she could...
4,madqueen,MadQueen,1128107891322380288,RT @HbWINGer27: A close up of Varys' letter ab...,en,Tue May 14 01:20:55 +0000 2019,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Neu,0.0,0.0,rt hbwinger27 close up of varys letter about d...


    # Remove all the special characters
    string = re.sub(r'\W', ' ', str(record["text"]))

    # remove all single characters
    record["Preprocess"] = re.sub(r'\s+[a-zA-Z]\s+', ' ', record["Preprocess"])

    # Remove single characters from the start
    record["Preprocess"] = re.sub(r'\^[a-zA-Z]\s+', ' ', record["Preprocess"]) 

    # Substituting multiple spaces with single space
    df["Preprocess"] = re.sub(r'\s+', ' ', df["Preprocess"], flags=re.I)

    # Removing prefixed 'b'
    df["Preprocess"] = re.sub(r'^b\s+', '', df["Preprocess"])

    # Converting to Lowercase
    df["Preprocess"] = df["Preprocess"].lower()

In [9]:
df_textpreprocess_backup = df

In [10]:
# Loop in the df to create test data which has atleast 1000 tweets per character/hashtag
Max_limit = 1000
char_list = ["JonSnow", "DaenerysTargaryen", "CerseiLannister", "AryaStark", "JaimeLannister", "TyrionLannister", "SansaStark", "BranStark", "BrienneOfTarth", "DavosSeaworth"]

for index, row in df.iterrows():
    test_preproc = re.sub(r'\W', ' ', str(row["text"]))

    comb_string = ' '.join(test_preproc)
    df.loc[index, "Preprocess"] = comb_string

In [ ]:
df = df_textpreprocess_backup
X = []
y = df["sentiment"]

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(df["Preprocess"]).toarray()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [12]:
classifier = RandomForestClassifier(n_estimators=500, random_state=0)  
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
y_pred = classifier.predict(X_test)  

In [14]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[5 1 1]
 [1 9 0]
 [2 1 0]]
             precision    recall  f1-score   support

        Neg       0.62      0.71      0.67         7
        Neu       0.82      0.90      0.86        10
        Pos       0.00      0.00      0.00         3

avg / total       0.63      0.70      0.66        20

0.7
